In [9]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import pandas as pd
import tensorflow as tf
import numpy as np
import pandas as pd

import os
import glob
import sys
import keras

In [2]:
df = pd.read_csv('../data/csv_raw/train_labels.csv')
df.sample(3)

,Unnamed: 0,row_id,tomo_id,Motor_axis_0,Motor_axis_1,Motor_axis_2,Array_shape_axis_0,Array_shape_axis_1,Array_shape_axis_2,Voxel_spacing,Number_of_motors
361,361,229,tomo_4ee35e,157.0,670.0,300.0,300,959,928,15.6,1
715,715,436,tomo_9a7701,166.0,258.0,87.0,500,924,956,19.7,6
466,466,260,tomo_5b34b2,112.0,613.0,233.0,300,960,928,13.1,1


In [3]:
def select_tomo_ids(df, number_of_slices=[300], number_of_motors=[0, 1], y_shape_range=(924, 960), x_shape_range=(924, 960)) -> pd.Series:
    '''
    Return the list of the tomo_ids obtained by filtering the DataFrame base on the given parameters

            Parameters:
                    df (pd.Dataframe): the dataset to filter
                    number_of_slices (list:int): number of slices per tomogram
                    max_number_of_motors (list:int): max number of motors
                    y_shape_range(tuple:int): tuple of the (min, max) y size of pictures
                    x_shape_range(tuple:int): tuple of the (min, max) x size of pictures

            Returns:
                    pd.Series: pandas Series of the tomo_ids corresponding to the filter
    '''
    df = df[(df['Array_shape_axis_1'] >= y_shape_range[0]) & (df['Array_shape_axis_2'] <= y_shape_range[1])]
    df = df[(df['Array_shape_axis_1'] >= x_shape_range[0]) & (df['Array_shape_axis_2'] <= x_shape_range[1])]
    df = df[(df['Array_shape_axis_0'].isin(number_of_slices)) & (df['Number_of_motors'].isin(number_of_motors))]


    return df.tomo_id


def selection_images_labels(df, dir_images, num_slices=[300], num_motors=[1]):

    ''''
    function to return the path to the selected images (which type, which tomos, how many motors,
    shape of the images)
    Parameters:
    ----------
    df = database (train)
    dir_images(str) = directory with the images we want to feed to the model
    num_slices, num_motors, y_shape_range, x_shape_range = params for the select_tomo_ids function

    Returns:
    -------
    filtered_image_paths (list or np.array): List of image paths.

    labels (np.array or list): Corresponding labels.
    '''

   # Step 1: Filter tomos
    tomo_ids = select_tomo_ids(df, number_of_slices=num_slices, number_of_motors=num_motors)
    df_select = df[df['tomo_id'].isin(tomo_ids)].copy()

    # Step 2: Set up labels
    df_select['motor_coord'] = df_select.apply(lambda row: (row['Motor_axis_2'], row['Motor_axis_1']), axis=1)

    # Step 3: Load all images
    dir_mean_image = f'../data/pictures_process/{dir_images}'
    all_images = glob.glob(os.path.join(dir_mean_image, '**', '*.jpg'), recursive=True)

    print(f"Found {len(all_images)} images in {dir_mean_image}")

    # Step 4: Match images using substring matching
    filtered_image_paths = []
    labels = []

    for _, row in df_select.iterrows():
        tomo_id = row['tomo_id']
        matched = [p for p in all_images if tomo_id in os.path.basename(p)]

        if matched:
            filtered_image_paths.append(matched[0])  # If multiple, take the first
            labels.append(row['Number_of_motors'])
        else:
            print(f"⚠️ No image found for tomo_id: {tomo_id}")

    print(f"Matched {len(filtered_image_paths)} image-label pairs")

    labels = np.array(labels, dtype=np.float32)
    return filtered_image_paths, labels


# Define image reading function
def read_img_jpg(path, label):
    """
    Reads a JPEG image from a file path, decodes it as a grayscale image (1 channel),
    normalizes pixel values to the range [0, 1], and returns it along with its label.

    Parameters:
    ----------
    path : tf.Tensor
        A scalar string tensor representing the file path to the JPEG image.

    label : tf.Tensor or any
        The label associated with the image (e.g., coordinates or class ID).

    Returns:
    -------
    img : tf.Tensor
        A 3D float32 tensor of shape (height, width, 1) representing the normalized image.

    label : same as input
        The original label passed in, unchanged.
    """
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=1)
    img = tf.cast(img, tf.float32) / 255.0  # normalize to [0, 1]
    return img, label



def batches_images_ram(
    read_img_jpg,
    filtered_image_paths,
    labels,
    shuffle=True,
    batch_size=32,
    split=False,
    val_fraction=0.2,
    test_fraction=0.2,
    seed=42
):
    """
    Load images and labels as tf.data.Dataset, optionally shuffle and batch,
    and optionally split into train/val/test datasets.

    Parameters:
    -----------
    read_img_jpg : function
        Function to load and preprocess image from path.

    filtered_image_paths : list or np.array
        List of image paths.

    labels : np.array or list
        Corresponding labels.

    shuffle : bool, default=True
        Whether to shuffle the dataset.

    batch_size : int, default=32
        Batch size.

    split : bool, default=False
        Whether to split dataset into train/val/test.

    val_fraction : float, default=0.2
        Fraction of data for validation.

    test_fraction : float, default=0.2
        Fraction of data for testing.

    seed : int, default=42
        Random seed for shuffling.

    Returns:
    --------
    If split=False:
        dataset : tf.data.Dataset
            Dataset with (image, label) pairs, batched and optionally shuffled.

    If split=True:
        train_ds, val_ds, test_ds : tf.data.Dataset
            The three splits, all batched and shuffled as specified.
    """

    dataset_size = len(filtered_image_paths)
    # Combine and optionally shuffle the data as a list of tuples
    data = list(zip(filtered_image_paths, labels))
    if shuffle:
        rng = np.random.default_rng(seed)
        rng.shuffle(data)

    # Unzip the shuffled data back
    filtered_image_paths, labels = zip(*data)

    # Convert back to lists or arrays
    filtered_image_paths = list(filtered_image_paths)
    labels = list(labels)

    if split:
        # Compute sizes
        val_size = int(val_fraction * dataset_size)
        test_size = int(test_fraction * dataset_size)
        train_size = dataset_size - val_size - test_size

        # Split into slices
        test_paths = filtered_image_paths[:test_size]
        print(test_paths)
        test_labels = labels[:test_size]

        val_paths = filtered_image_paths[test_size:test_size + val_size]
        print(val_paths)
        val_labels = labels[test_size:test_size + val_size]

        train_paths = filtered_image_paths[test_size + val_size:]
        print(train_paths)
        train_labels = labels[test_size + val_size:]

        # Create tf.data.Dataset for each
        train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels)).map(read_img_jpg).batch(batch_size)
        val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels)).map(read_img_jpg).batch(batch_size)
        test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels)).map(read_img_jpg).batch(batch_size)

        return train_ds, val_ds, test_ds #, test_paths, test_labels

    else:
        # Single dataset
        dataset = tf.data.Dataset.from_tensor_slices((filtered_image_paths, labels))
        dataset = dataset.map(read_img_jpg).batch(batch_size)
        return dataset, filtered_image_paths, labels


def plot_history_regression(history, xlims=None, ylims=None):
    '''
    Function to plot learning curves for a regression task
    Parameters:
        history: output from a model.fit
        xlims,ylims (tuple of float, optional): limits for x and y axes, if not
                                                provided are defined as
                                                (0,max(epochs)), (0,max(loss))
    '''
    if xlims is None:
        xlims = (0, max(history.epoch))
    if ylims is None:
        ylims = (0, max(history.history['loss']))
    print(xlims,ylims)
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[0].set_ylim(ylims)
    ax[0].set_xlim(xlims)
    ax[0].set_xlabel('Epochs')
    ax[0].set_ylabel('Loss')

    ax[1].set_title('recall loss')
    ax[1].plot(history.epoch, history.history["recall"], label="Train recall")
    ax[1].plot(history.epoch, history.history["val_recall"], label="Validation recall")
    ax[1].set_ylim(ylims)
    ax[1].set_xlim(xlims)
    ax[1].set_xlabel('Epochs')
    ax[1].set_ylabel('recall')
    ax[0].legend()
    ax[1].legend()


In [4]:
def generate_base_data():
  all_slices_number = df['Array_shape_axis_0'].unique()

  filtered_image_paths,labels = selection_images_labels(df, 'adaptequal_1_padded', num_slices=list(all_slices_number), num_motors=[0, 1])

  train_ds, val_ds, test_ds = batches_images_ram(
      read_img_jpg,
      filtered_image_paths,
      labels,
      shuffle=True,
      batch_size=32,
      split=True,
      val_fraction=0.2,
      test_fraction=0.2,
      seed=42)

  return train_ds, val_ds, test_ds

In [5]:
train_ds, val_ds, test_ds = generate_base_data()

Found 648 images in ../data/pictures_process/adaptequal_1_padded
Matched 578 image-label pairs
['../data/pictures_process/adaptequal_1_padded/tomo_dae195.jpg', '../data/pictures_process/adaptequal_1_padded/tomo_f2fa4a.jpg', '../data/pictures_process/adaptequal_1_padded/tomo_cabaa0.jpg', '../data/pictures_process/adaptequal_1_padded/tomo_f7f28b.jpg', '../data/pictures_process/adaptequal_1_padded/tomo_ed1c97.jpg', '../data/pictures_process/adaptequal_1_padded/tomo_ff505c.jpg', '../data/pictures_process/adaptequal_1_padded/tomo_8f4d60.jpg', '../data/pictures_process/adaptequal_1_padded/tomo_2aeb29.jpg', '../data/pictures_process/adaptequal_1_padded/tomo_651ecd.jpg', '../data/pictures_process/adaptequal_1_padded/tomo_e96200.jpg', '../data/pictures_process/adaptequal_1_padded/tomo_0d4c9e.jpg', '../data/pictures_process/adaptequal_1_padded/tomo_2dcd5c.jpg', '../data/pictures_process/adaptequal_1_padded/tomo_983fce.jpg', '../data/pictures_process/adaptequal_1_padded/tomo_7b1ee3.jpg', '../data

2025-06-10 10:20:29.958469: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-06-10 10:20:29.959405: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2025-06-10 10:20:29.959500: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ADMINIS-I3M0BV4): /proc/driver/nvidia/version does not exist
2025-06-10 10:20:29.964534: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import os
import numpy as np
import pandas as pd
import keras
import matplotlib.image as img
import tensorflow as tf

MODEL_CLASSIF_URI = 'https://storage.cloud.google.com/models-wagon1992-group-project/7/06ba2fef2e314aba94bdf6286c0440cc/artifacts/model/data/model.keras'
#MODEL_X_Y_URI =
#MODEL_Z_URI =

test_tomos = ['tomo_dae195', 'tomo_f2fa4a', 'tomo_cabaa0', 'tomo_f7f28b', 'tomo_ed1c97', 'tomo_ff505c', 'tomo_8f4d60', 'tomo_2aeb29', 'tomo_651ecd', 'tomo_e96200', 'tomo_0d4c9e', 'tomo_2dcd5c', 'tomo_983fce', 'tomo_7b1ee3', 'tomo_8b6795', 'tomo_dcb9b4', 'tomo_e764a7', 'tomo_e26c6b', 'tomo_331130', 'tomo_f8b835', 'tomo_746d88', 'tomo_9cd09e', 'tomo_b9eb9a', 'tomo_cf0875', 'tomo_7cf523', 'tomo_fd41c4', 'tomo_54e1a7', 'tomo_ca472a', 'tomo_6478e5', 'tomo_e9b7f2', 'tomo_247826', 'tomo_675583', 'tomo_f0adfc', 'tomo_378f43', 'tomo_19a313', 'tomo_172f08', 'tomo_f3e449', 'tomo_3b83c7', 'tomo_8c13d9', 'tomo_2c607f', 'tomo_c11e12', 'tomo_412d88', 'tomo_4b124b', 'tomo_38c2a6', 'tomo_ec1314', 'tomo_1c38fd', 'tomo_e63ab4', 'tomo_f07244', 'tomo_210371', 'tomo_d6e3c7', 'tomo_935f8a', 'tomo_a4c52f', 'tomo_a46b26', 'tomo_fadbe2', 'tomo_b28579', 'tomo_35ec84', 'tomo_369cce', 'tomo_6c203d', 'tomo_b80310', 'tomo_640a74', 'tomo_22976c', 'tomo_d21396', 'tomo_ecbc12', 'tomo_040b80', 'tomo_85708b', 'tomo_b98cf6', 'tomo_e1e5d3', 'tomo_138018', 'tomo_3264bc', 'tomo_e50f04', 'tomo_d723cd', 'tomo_2a6ca2', 'tomo_1f0e78', 'tomo_67565e', 'tomo_fd5b38', 'tomo_05b39c', 'tomo_372a5c', 'tomo_c3619a', 'tomo_ba76d8', 'tomo_a67e9f', 'tomo_a6646f', 'tomo_db656f', 'tomo_4102f1', 'tomo_bb5ac1', 'tomo_4ed9de', 'tomo_61e947', 'tomo_1da0da', 'tomo_821255', 'tomo_3e7783', 'tomo_c84b46', 'tomo_974fd4', 'tomo_444829', 'tomo_b50c0f', 'tomo_2a6091', 'tomo_fa5d78', 'tomo_bdd3a0', 'tomo_1c2534', 'tomo_d916dc', 'tomo_bdc097', 'tomo_7036ee', 'tomo_cacb75', 'tomo_5b359d', 'tomo_7fa3b1', 'tomo_049310', 'tomo_dd36c9', 'tomo_e3864f', 'tomo_0a8f05', 'tomo_ff7c20', 'tomo_0fab19', 'tomo_1c75ac', 'tomo_d0699e', 'tomo_1e9980', 'tomo_4ee35e', 'tomo_6943e6', 'tomo_99a3ce']

df = pd.read_csv('../data/csv_raw/train_labels.csv')

df_test = df[df['tomo_id'].isin(test_tomos)]

path = f'../data/pictures_process/adaptequal_1_padded'

X_test = [img.imread(f'{path}/{tomo}.jpg')/255 for tomo in test_tomos]
y_test = df_test.Number_of_motors

df_test

,Unnamed: 0,row_id,tomo_id,Motor_axis_0,Motor_axis_1,Motor_axis_2,Array_shape_axis_0,Array_shape_axis_1,Array_shape_axis_2,Voxel_spacing,Number_of_motors
0,0,16,tomo_049310,-1.0,-1.0,-1.0,500,924,956,19.7,0
29,29,587,tomo_cabaa0,-1.0,-1.0,-1.0,500,924,956,19.7,0
30,30,648,tomo_dd36c9,-1.0,-1.0,-1.0,500,924,956,16.1,0
35,35,77,tomo_1c2534,-1.0,-1.0,-1.0,800,927,959,13.1,0
36,36,126,tomo_2aeb29,-1.0,-1.0,-1.0,800,927,959,13.1,0
...,...,...,...,...,...,...,...,...,...,...,...
583,583,656,tomo_e1e5d3,21.0,564.0,452.0,300,960,928,13.1,1
586,586,674,tomo_e63ab4,157.0,415.0,265.0,300,960,928,13.1,1
591,591,699,tomo_f0adfc,175.0,550.0,671.0,300,960,928,13.1,1
592,592,701,tomo_f2fa4a,59.0,434.0,600.0,300,960,928,15.6,1


In [7]:
####### Motor Detection #######
classif_model = keras.saving.load_model('../models/classif.keras') # changer le nom

preds = []

for X in X_test:
    X = X[:, :, 0]
    X = tf.expand_dims(X, axis=-1)
    X = tf.expand_dims(X, axis=0)
    preds.append(float(classif_model.predict(X)))

df_test['Motor_pred'] = preds
df_test

2025-06-09 19:11:42.138316: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f7a1c017660 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-06-09 19:11:42.138399: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Host, Default Version
2025-06-09 19:11:42.194706: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-06-09 19:11:44.026978: I tensorflow/compiler/jit/xla_compilation_cache.cc:476] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 

/tmp/ipykernel_7223/710696536.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Motor_pred'] = preds


,Unnamed: 0,row_id,tomo_id,Motor_axis_0,Motor_axis_1,Motor_axis_2,Array_shape_axis_0,Array_shape_axis_1,Array_shape_axis_2,Voxel_spacing,Number_of_motors,Motor_pred
0,0,16,tomo_049310,-1.0,-1.0,-1.0,500,924,956,19.7,0,0.035605
29,29,587,tomo_cabaa0,-1.0,-1.0,-1.0,500,924,956,19.7,0,0.520573
30,30,648,tomo_dd36c9,-1.0,-1.0,-1.0,500,924,956,16.1,0,0.175888
35,35,77,tomo_1c2534,-1.0,-1.0,-1.0,800,927,959,13.1,0,0.864853
36,36,126,tomo_2aeb29,-1.0,-1.0,-1.0,800,927,959,13.1,0,0.244324
...,...,...,...,...,...,...,...,...,...,...,...,...
583,583,656,tomo_e1e5d3,21.0,564.0,452.0,300,960,928,13.1,1,0.627805
586,586,674,tomo_e63ab4,157.0,415.0,265.0,300,960,928,13.1,1,0.236472
591,591,699,tomo_f0adfc,175.0,550.0,671.0,300,960,928,13.1,1,0.647547
592,592,701,tomo_f2fa4a,59.0,434.0,600.0,300,960,928,15.6,1,0.457311


In [8]:
df_regression = df_test[df_test['Motor_pred'] >= 0.50]
df_regression

,Unnamed: 0,row_id,tomo_id,Motor_axis_0,Motor_axis_1,Motor_axis_2,Array_shape_axis_0,Array_shape_axis_1,Array_shape_axis_2,Voxel_spacing,Number_of_motors,Motor_pred
29,29,587,tomo_cabaa0,-1.0,-1.0,-1.0,500,924,956,19.7,0,0.520573
35,35,77,tomo_1c2534,-1.0,-1.0,-1.0,800,927,959,13.1,0,0.864853
40,40,510,tomo_b28579,-1.0,-1.0,-1.0,800,927,959,13.1,0,0.977297
45,45,15,tomo_040b80,-1.0,-1.0,-1.0,600,928,960,16.1,0,0.794395
51,51,65,tomo_172f08,-1.0,-1.0,-1.0,800,928,960,13.1,0,0.914957
...,...,...,...,...,...,...,...,...,...,...,...,...
577,577,632,tomo_d916dc,163.0,524.0,365.0,300,960,928,16.8,1,0.744503
580,580,647,tomo_dcb9b4,153.0,493.0,359.0,300,960,928,15.6,1,0.801574
583,583,656,tomo_e1e5d3,21.0,564.0,452.0,300,960,928,13.1,1,0.627805
591,591,699,tomo_f0adfc,175.0,550.0,671.0,300,960,928,13.1,1,0.647547


In [9]:
df_regression.shape

(70, 12)

In [10]:
classif_model = keras.saving.load_model('../models/classif.keras') # changer le nom

In [11]:
from sklearn.metrics import classification_report

y_pred_labels = [int(pred > 0.5) for pred in preds]

print(classification_report(y_pred_labels, y_test))

              precision    recall  f1-score   support

           0       0.33      0.40      0.36        45
           1       0.55      0.47      0.51        70

    accuracy                           0.44       115
   macro avg       0.44      0.44      0.43       115
weighted avg       0.46      0.44      0.45       115



In [12]:
y_test

0      0
29     0
30     0
35     0
36     0
      ..
583    1
586    1
591    1
592    1
598    1
Name: Number_of_motors, Length: 115, dtype: int64

In [13]:
X_test

[array([[[0.75294118, 0.75294118, 0.75294118],
         [0.75294118, 0.75294118, 0.75294118],
         [0.75294118, 0.75294118, 0.75294118],
         ...,
         [0.51764706, 0.51764706, 0.51764706],
         [0.51764706, 0.51764706, 0.51764706],
         [0.51764706, 0.51764706, 0.51764706]],
 
        [[0.75294118, 0.75294118, 0.75294118],
         [0.75294118, 0.75294118, 0.75294118],
         [0.75294118, 0.75294118, 0.75294118],
         ...,
         [0.51764706, 0.51764706, 0.51764706],
         [0.51764706, 0.51764706, 0.51764706],
         [0.51764706, 0.51764706, 0.51764706]],
 
        [[0.75294118, 0.75294118, 0.75294118],
         [0.75294118, 0.75294118, 0.75294118],
         [0.75294118, 0.75294118, 0.75294118],
         ...,
         [0.51764706, 0.51764706, 0.51764706],
         [0.51764706, 0.51764706, 0.51764706],
         [0.51764706, 0.51764706, 0.51764706]],
 
        ...,
 
        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.     

In [14]:
#!wget 'https://storage.cloud.google.com/models-wagon1992-group-project/7/06ba2fef2e314aba94bdf6286c0440cc/artifacts/model/data/model.keras' -O classif.keras

In [6]:
X_test = []
y_test = []

for batch_x, batch_y in test_ds:
    X_test.append(batch_x.numpy())
    y_test.append(batch_y.numpy())

# Fusionner les batches
X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

In [ ]:
classif_model.predict(X_test[0])

2025-06-10 10:27:42.738782: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x56189780b990 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-06-10 10:27:42.738929: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Host, Default Version
2025-06-10 10:27:43.102134: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-06-10 10:27:51.786979: I tensorflow/compiler/jit/xla_compilation_cache.cc:476] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 1/30 ━━━━━━━━━━━━━━━━━━━━ 1:09:02 143s/step

In [ ]:
classif_model.predict(X_test)

In [16]:
from sklearn.metrics import classification_report

y_pred = classif_model.predict(X_test)

y_pred_labels = (y_pred > 0.5).astype(int)

print(classification_report(y_pred_labels, y_test))

4/4 ━━━━━━━━━━━━━━━━━━━━ 128s 30s/step
              precision    recall  f1-score   support

           0       0.75      0.91      0.82        45
           1       0.93      0.80      0.86        70

    accuracy                           0.84       115
   macro avg       0.84      0.86      0.84       115
weighted avg       0.86      0.84      0.85       115

